# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [57]:
import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from sklearn.model_selection import train_test_split
import pandas as pd
import csv


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [58]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.subscription_id, sep = '\n')

quick-starts-ws-131714
aml-quickstarts-131714
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


In [59]:
# choose a name for experiment
experiment_name = 'capstone-twitter-exp'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-twitter-exp,quick-starts-ws-131714,Link to Azure Machine Learning studio,Link to Documentation


In [60]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"

try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
    
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',vm_priority='dedicated',min_nodes=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True)         
    print("Cluster is ready")

Existing cluster detected, make use of it!


In [61]:
 # Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Twitter-Data"
description_text = "External dataset from kaggle"

if key in ws.datasets.keys(): 
        found = True
        print('DATASET IS REGISTERED')
        dataset = ws.datasets[key] 
        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

DATASET IS REGISTERED


,fav_number,retweet_count,tweet_count,tweet_id
count,8.640000e+02,1.217000e+03,588.000000,2.070000e+03
mean,9.850073e+05,6.706329e+06,25.571429,3.182319e+14
std,2.775027e+07,7.366945e+07,539.339415,1.447868e+16
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,2.732250e+03,4.000000e+00,1.000000,0.000000e+00
50%,1.079300e+04,2.060000e+02,1.000000,0.000000e+00
75%,3.575825e+04,2.732000e+03,1.000000,1.000000e+00
max,8.157222e+08,8.157394e+08,13030.000000,6.587400e+17


In [62]:
dataset.take(5).to_pandas_dataframe() 

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Perkins, Yahoo!, Sprint PCS, AirTouch, Air Force. Stanford GSB, UVa. Dad, Husband, Brother. Golfer.",202,Column113,0084B4_30,douggarland,Column116,https://pbs.twimg.com/profile_images/259703936/IMG_8444_normal.JPG,0_31,C0DEED_32
0,"#Islam #Muslim #muslims""",None,52230,10/26/15 12:40,6.5873E+17,ksa,Pacific Time (US & Canada)\r815719253,FALSE,finalized,3,...,3,10/26/15 23:12,male,1,yes,1,12/5/12 6:18,"""Award Winning Author of the paranormal Romanc...",Hath No Fury technology writer,None
1,"http://t.co/LmMiOwenQQ #ASMSG""",103,None,DD2E44,HalRappaport,None,https://pbs.twimg.com/profile_images/363652832...,0,FFFFFF,All the #magic in Hath No FURY is based on REA...,...,None,None,None,None,None,None,None,None,None,None
2,Me November 1st: it's my birthday month,"bow down to me""",None,4965,10/26/15 12:40,6.5873E+17,"Algonac & Mt. Pleasant, MI",Eastern Time (US & Canada)\r815719259,FALSE,finalized,...,None,None,None,None,None,None,None,None,None,None
3,"All I Kno Is Grind Wea I Come From ‰Ïö""",49,None,0084B4,300Stain,None,https://pbs.twimg.com/profile_images/656370495...,0,FFFFFF,Did Alot Up In The Past Ion Wont Back,...,None,None,None,None,None,None,None,None,None,None
4,capri eternal...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [63]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             experiment_timeout_minutes=20,
                             primary_metric='AUC_weighted',
                             n_cross_validations=5,
                             max_concurrent_iterations=4,
                             max_cores_per_iteration=4,
                             compute_target=new_cluster,
                             label_column_name='gender',
                             featurization='auto',
                             enable_voting_ensemble=True
)

In [64]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [68]:
from azureml.pipeline.steps import AutoMLStep

automl_step = AutoMLStep(
    name='remote_run',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)


In [70]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

ConfigException: ConfigException:
	Message: Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

In [40]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

ConfigException: ConfigException:
	Message: Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service